In [1]:
import tensorflow as tf
import numpy as np

sess = tf.InteractiveSession()
x = np.array([[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,1,1,0],[0,1,1,0,0]],dtype = np.float32)
w = np.array([[1,0,1],[0,1,0],[1,0,1]],dtype = np.float32)
x = tf.convert_to_tensor(x)
w = tf.convert_to_tensor(w)
xx = tf.reshape(x,[1,5,5,1])
ww = tf.reshape(w,[3,3,1,1])
print('x:[1,5,5,1]')
print(x.eval())
print('w:[3,3,1,1]')
print(w.eval())

conv_same = tf.nn.conv2d(xx,ww,strides = [1,1,1,1],padding = 'SAME')
conv_same = tf.reshape(conv_same,[5,5])
print('conv_same:[1,5,5,1]')
print(conv_same.eval())

conv_valid = tf.nn.conv2d(xx,ww,strides = [1,1,1,1],padding = 'VALID')
conv_valid = tf.reshape(conv_valid,[3,3])
print('conv_valid:[1,3,3,1]')
print(conv_valid.eval())


x = np.array([[1,1,2,4],[5,6,7,8],[3,2,1,0],[1,2,3,4]],dtype=np.float32)
x = tf.convert_to_tensor(x)
print('x:[1,4,4,1]')
print(x.eval())
x = tf.reshape(x,[1,4,4,1])
pool = tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
pool = tf.reshape(pool,[2,2])
print('pool:[1,2,2,1] ------------2 2   ，2  2-------')
print(pool.eval())

pool1 = tf.nn.max_pool(x,ksize = [1,3,3,1],strides = [1,2,2,1],padding = 'SAME')
pool1 = tf.reshape(pool,[2,2])
print('pool1:[1,2,2,1] ------------3 3   ，  2  2-------')
print(pool1.eval())

x:[1,5,5,1]
[[ 1.  1.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  1.  1.  0.  0.]]
w:[3,3,1,1]
[[ 1.  0.  1.]
 [ 0.  1.  0.]
 [ 1.  0.  1.]]
conv_same:[1,5,5,1]
[[ 2.  2.  3.  1.  1.]
 [ 1.  4.  3.  4.  1.]
 [ 1.  2.  4.  3.  3.]
 [ 1.  2.  3.  4.  1.]
 [ 0.  2.  2.  1.  1.]]
conv_valid:[1,3,3,1]
[[ 4.  3.  4.]
 [ 2.  4.  3.]
 [ 2.  3.  4.]]
x:[1,4,4,1]
[[ 1.  1.  2.  4.]
 [ 5.  6.  7.  8.]
 [ 3.  2.  1.  0.]
 [ 1.  2.  3.  4.]]
pool:[1,2,2,1] ------------2 2   ，2  2-------
[[ 6.  8.]
 [ 3.  4.]]
pool1:[1,2,2,1] ------------3 3   ，  2  2-------
[[ 6.  8.]
 [ 3.  4.]]


In [1]:
#%%
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()


def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
  
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')  
                        
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1,28,28,1])
                        
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.global_variables_initializer().run()
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.1
step 100, training accuracy 0.88
step 200, training accuracy 0.92
step 300, training accuracy 0.92
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.86
step 700, training accuracy 0.9
step 800, training accuracy 0.94
step 900, training accuracy 1
step 1000, training accuracy 0.92
step 1100, training accuracy 0.96
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 0.94
step 1500, training accuracy 0.96
step 1600, training accur

KeyboardInterrupt: 